## Apply Deep Learning Transformer Architecture



## Load Libraries

In [ ]:
!pip -q install mlflow

In [ ]:
import os
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from tokenizers import BertWordPieceTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFDistilBertModel

import mlflow
import mlflow.tensorflow


import matplotlib.pyplot as plt

## Load Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("/content/drive/Othercomputers/My Mac/Documents/datascience_projects/my_kedro_sklearn_nlp/toxic-comment-classification-kedro/notebooks")

In [ ]:
train_df = pd.read_csv("../data/01_raw/jigsaw-toxic-comment-train.csv")
val_df = pd.read_csv("../data/01_raw/validation.csv")
test_df = pd.read_csv("../data/01_raw/test.csv")

## Preprocessing - Tokenizer

In [ ]:
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Hyperparameters setups
EPOCHS=15
BATCH_SIZE= 512
MAX_LEN = 512

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("../data/05_model_input")

In [ ]:
fast_tokenizer = BertWordPieceTokenizer("../data/05_model_input/vocab.txt")
fast_tokenizer

In [ ]:

def encoded(texts: list, tokenizer: BertWordPieceTokenizer, chunk_size=256, maxlen=512)-> np.array:
    """

    Args:
        texts (list): all the input samples that will be used for tokenization
        tokenizer (BertWordPieceTokenizer): Distil-BERT tokenizer
        chunk_size (int, optional): The batch_size that will be passed for tokenization. Defaults to 256.
        maxlen (int, optional): Dimensions of the embeddings returned in the tokenization process. Defaults to 512.

    Returns:
        np.array: the array_id returned after preprocess
    """

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].to_list()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])

    return np.array(all_ids)

In [ ]:
X_train = encoded(train_df.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_train = train_df.toxic.values
X_val = encoded(val_df.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_val = val_df.toxic.values
X_test = encoded(test_df.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

In [ ]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (X_train, y_train)
    ).repeat(

    ).shuffle(
        1000
    ).batch(
        BATCH_SIZE
    ).prefetch(
        AUTO
    )
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

## Build DL Transformer Model & Start Training

In [ ]:
def f1_score(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """Compute the F1 score for binary classification

    Args:
        y_true (tf.Tensor): Ground truth labels, a tensor containing binary values (0 or 1).
        y_pred (tf.Tensor): Predicted probabilities, a tensor with values in the range [0, 1]

    Returns:
        tf.Tensor: The F1 score computed as the harmonic mean of precision and recall (sentivity)
    """

    y_pred = tf.round(y_pred)
    y_true = tf.cast(y_true, tf.float32)
    tp = tf.reduce_sum(
        tf.cast(
            y_true * y_pred, tf.float32
        )
    )
    fp = tf.reduce_sum(
        tf.cast(
            (1 - y_true) * y_pred, tf.float32
        )
    )
    fn = tf.reduce_sum(
        tf.cast(
            y_true * (1 - y_pred), tf.float32
        )
    )

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1



In [ ]:
class DistilBertLayer(Layer):
    def __init__(self, **kwargs):
        super(DistilBertLayer, self).__init__(**kwargs)
        self.transformer_layer = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

    def call(self, inputs):
        # Forward pass through DistilBERT
        output = self.transformer_layer(input_ids=inputs)[0]  # Sequence output
        return output

def build_model(max_len=512) -> Model:
    """Function for get the BERT training model

    Args:
        transformer (TFDistilBertModel): Pretrained model
        max_len (int, optional): Number of dimensions of the last layer of pretrained model. Defaults to 512.

    Returns:
        Model: Keras Model compiled
    """
    input_word_ids = Input(
        shape=(max_len,),
        dtype=tf.int32,
        name="input_word_ids"
    )

    sequence_output = DistilBertLayer()(input_word_ids)

    cls_token = sequence_output[:, 0, :]

    out = Dense(1, activation="sigmoid")(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [ ]:
model = build_model(MAX_LEN)

model.summary()

In [ ]:
def plot_training_history_with_mlflow(history, metrics=("accuracy", "loss"), log_to_mlflow=True):
    """
    Plots training and validation metrics for a given training history and logs to MLflow.

    Args:
        history (dict): Dictionary containing training history (e.g., history.history).
        metrics (tuple): Tuple of metrics to plot (default: ("accuracy", "loss")).
        log_to_mlflow (bool): If True, logs the plots to MLflow.
    """
    plt.figure(figsize=(12, 6))

    for i, metric in enumerate(metrics):
        plt.subplot(1, len(metrics), i + 1)
        train_metric = history.get(metric)
        val_metric = history.get(f"val_{metric}")

        if train_metric:
            plt.plot(train_metric, label=f'Train {metric.capitalize()}')
        if val_metric:
            plt.plot(val_metric, label=f'Validation {metric.capitalize()}')

        plt.title(f'{metric.capitalize()} Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)

    plt.tight_layout()

    # Show plot
    plt.show()

    # Log plot to MLflow
    if log_to_mlflow:
        for metric in metrics:
            plot_path = f"../data/09_tracking/{metric}_plot.png"
            plt.figure(figsize=(6, 4))

            train_metric = history.get(metric)
            val_metric = history.get(f"val_{metric}")

            if train_metric:
                plt.plot(train_metric, label=f'Train {metric.capitalize()}')
            if val_metric:
                plt.plot(val_metric, label=f'Validation {metric.capitalize()}')

            plt.title(f'{metric.capitalize()} Over Epochs')
            plt.xlabel('Epoch')
            plt.ylabel(metric.capitalize())
            plt.legend()
            plt.grid(True)

            plt.tight_layout()
            plt.savefig(plot_path)
            mlflow.log_artifact(plot_path)
            plt.close()


def train_log_with_mlflow():
    mlflow.set_tracking_uri("../mlruns")
    mlflow.set_experiment(
        "Applying Kedro Orchestration Pipelines to Deploy a Deep Learning Transformer Architecture for the Toxic Comment Classification Problem"
    )

    with mlflow.start_run():
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("learning_rate", 1e-5)
        mlflow.log_param("num_train_samples", X_train.shape[0])
        mlflow.log_param("num_val_samples", X_val.shape[0])

        mlflow.tensorflow.autolog()

        n_steps = X_train.shape[0] // BATCH_SIZE

        train_history = model.fit(
            train_dataset,
            steps_per_epoch=n_steps,
            validation_data=val_dataset,
            epochs=EPOCHS

        )


        plot_training_history_with_mlflow(train_history.history)

        for epoch, accuracy in enumerate(train_history.history["accuracy"]):
            mlflow.log_metric(f"train_accuracy_epoch_{epoch}", accuracy)

        for epoch, accuracy in enumerate(train_history.history["val_accuracy"]):
            mlflow.log_metric(f"val_accuracy_epoch_{epoch}", accuracy)

        mlflow.tensorflow.log_model(model, artifact_path="distilbert_model", registered_model_name="DistilBERT_Toxic_Comment_Classification")

        with open("../data/07_model_output/model_summary.txt", "w") as f:
            model.summary(print_fn=lambda x: f.write(x + "\n"))
        mlflow.log_artifact("../data/07_model_output/model_summary.txt")

        print(f"RUN ID: {mlflow.active_run().info.run_id}")



In [ ]:
train_log_with_mlflow()

In [ ]:
model.save("../data/06_models/toxic_comment_classification_model.keras")

In [ ]:
mlflow.tensorflow.log_model(
    model,
    artifact_path="toxic_comment_classification_model"
)